# Digital Education - Learner Modeling Lecture

### BKT Training & Prediction Notebook

Inspired by content from EPFL's [Machine Learning for Behavioral Data](https://github.com/epfl-ml4ed/mlbd-2023) course.   
**Date**: November 9, 2025


**Overview**: In this tutorial, we introduce a Bayesian Knowledge Tracing (BKT) model to represent students' acquiry of knowledge through problem solving. This model is used in many intelligent tutoring systems (ITS) as a way to estimate what skills students have mastered and what skills they need improvement on. 

We will train the BKT model from scratch and examine the behavior of the model and of students across different skills through their BKT learning curves. To do this, we will use data from a popular, real-world learning platform called ASSISTments.

**Dataset**: ASSISTments is a free tool for assigning and assessing math problems and homework. Teachers can select and assign problem sets. Once they get an assignment, students can complete it at their own pace and with the help of hints, multiple chances, and immediate feedback. Teachers get instant results broken down by individual student or for the whole class. The dataset involves 4,217 middle-school students practicing an electronic tutor that teaches and evaluates students in grade-school math, with a total of 525,534 trials. The student data are in a comma-delimited text file with one row per trial. The columns should correspond to a trial's user id, the order id (timestamp), the skill name, and and whether the student produced a correct response in the trial. More information on the platform can be found [here](https://www.commonsense.org/education/website/assistments). 

The ASSISTments data sets are often used for benchmarking knowledge tracing models. We will play with a simplified data set that contains the following columns:

| Name                   | Description                         |
| ---------------------- | ------------------------------------------------------------ |
| user_id | The ID of the student who is solving the problem.  | |
| order_id | The temporal ID (timestamp) associated with the student's answer to the problem.  | |
| skill_name | The name of the skill associated with the problem. | |
| correct | The student's performance on the problem: 1 if the problem's answer is correct at the first attempt, 0 otherwise. 

We first load the data set.

In [ ]:
# Principal package imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sc

# Scikit-learn package imports
from sklearn import feature_extraction, model_selection
from sklearn.metrics import root_mean_squared_error, roc_auc_score

# Monkey-patch the function before importing the module
import random
old_randint = random.randint
random.randint = lambda a, b: old_randint(a, int(b))

# Now import the package
from pyBKT.models import Model

# Restore the original randint function to avoid side effects
random.randint = old_randint

In [ ]:
assistments = pd.read_csv('assistments.csv', low_memory=False).dropna()
assistments.head()

Next, we print the number of unique students and skills in this data set.

In [ ]:
print("Number of unique students in the dataset:", len(set(assistments['user_id'])))
print("Number of unique skills in the dataset:", len(set(assistments['skill_name'])))

To keep things simpler for demonstration purposes, we will focus on the following 6 skills in this lecture:  
`'Circle Graph', 'Venn Diagram', 'Mode', 'Division Fractions', 'Finding Percents', 'Area Rectangle'`

In [ ]:
skills_subset = ['Circle Graph', 'Venn Diagram', 'Mode', 'Division Fractions', 'Finding Percents', 'Area Rectangle']
data = assistments[assistments['skill_name'].isin(skills_subset)]

print("Skill set:", set(data['skill_name']))
print("Number of unique students in the subset:", len(set(data['user_id'])))
print("Number of unique skills in the subset:", len(set(data['skill_name'])))

## BKT Model - Training & Prediction

We will use a train-test setting (20% of students in the test set). The `create_iterator` function creates an iterator object able to split student's interactions included in data in 10 folds such that the same student does not appear in two different folds. To do so, we appropriately initialize a scikit-learn's GroupShuffleSplit iterator with 80% training set size and non-overlapping groups, then return the iterator.

In [ ]:
def create_iterator(data):
    '''
    Create an iterator to split interactions in data into train and test, with the same student not appearing in two diverse folds.
    :param data:        Dataframe with student's interactions.
    :return:            An iterator.
    '''    
    # Both passing a matrix with the raw data or just an array of indexes works
    X = np.arange(len(data.index)) 
    # Groups of interactions are identified by the user id (we do not want the same user appearing in two folds)
    groups = data['user_id'].values 
    return model_selection.GroupShuffleSplit(n_splits=1, train_size=.8, test_size=0.2, random_state=0).split(X, groups=groups)

Let's check the output of this function and a few properties of the iterator. 

In [ ]:
tested_user_ids = set()
for skill in skills_subset:
    print("--", skill, "--")
    skill_data = data[data['skill_name'] == skill]
    for iteration, (train_index, test_index) in enumerate(create_iterator(skill_data)):
        # There should only be one iteration per skill, because we only specify one fold for the split. 
        # If we wanted multiple folds, we could expand the n_splits parameter in create_iterator.
        user_ids = skill_data['user_id'].unique()
        train_user_ids = skill_data.iloc[train_index]['user_id'].unique()
        test_user_ids = skill_data.iloc[test_index]['user_id'].unique()
        print('Iteration:', iteration)
        print('Intersection between train and test user IDs:', set(train_user_ids) & set(test_user_ids))
        print('All user IDs in train and test user union:', len(set(train_user_ids).union(set(test_user_ids))) == len(user_ids))
        print('User IDs tested more than once:', set(tested_user_ids) & set(test_user_ids))
        print('')

In our split, no user appears in both training and test sets. The union of the users in both training and test sets given us the full set of user ids in the dataset. Each user appears in the test set exactly once.  

Next, we train a BKT model for each skill on the training data set and then predict on the test data set.
We obtain `df_preds`, a data frame containing the predictions for each user and skill in the test data set, and `df_param`, a data frame containing the model parameters for each skill.

In [ ]:
df_preds = pd.DataFrame()
df_params = pd.DataFrame()
# Train a BKT model for each skill
for skill in skills_subset:
    print("--", skill, "--")
    skill_data = data[data['skill_name'] == skill]
    for iteration, (train_index, test_index) in enumerate(create_iterator(skill_data)):
        # Split data in training and test sets
        X_train, X_test = skill_data.iloc[train_index], skill_data.iloc[test_index]
        # Initialize and fit the model
        model = Model(seed=0)
        %time model.fit(data=X_train) 
        # Compute predictions
        preds = model.predict(data=X_test)[['user_id', 'skill_name', 'correct', 'correct_predictions']]
        df_preds = pd.concat([df_preds, preds])
        # Store the model parameters
        params = model.params()
        df_params = pd.concat([df_params, params])
        
# Print the the resulting dataframe
display(df_preds)

### Task 1: Model predictions for selected skills and students

In a first step, we explore how our models behave for selected skills and students. We will first write the inference function based on the formulas seen in the lecture.

In [ ]:
def bkt_predict(p_0, p_s, p_g, p_l, p_f,o):
#p: BKT parameters
#o: observations o_0,...,o_T-1
#return: vector ps (ps_t = 1|o_0,...,o_t-1) for t=0,...,t=T
#return: vector po (po_t = 1|o_0,...,o_t-1) for t=0,...,t=T

    T = len(o)+1
    ps = np.zeros(T)
    po = np.zeros(T)

    #time step 0
    ps[0] = p_0
    po[0] = (1-p_s)*p_0 + p_g*(1-p_0)
    
    #time steps 1,...,T
    for t in range(1,T):
        
        #compute posterior at time step t-1
        if o[t-1] == 1:
            post = (1-p_s)*ps[t-1]/((1-p_s)*ps[t-1] + p_g*(1-ps[t-1]))
        else:
            post = p_s*ps[t-1]/(p_s*ps[t-1] + (1-p_g)*(1-ps[t-1]))
            
        #update state at t
        ps[t] = (1-p_f)*post + p_l*(1-post)
        
        #predict observation at t
        po[t] = (1-p_s)*ps[t] + p_g*(1-ps[t])

    #return predictions
    return ps, po



We can now get the parameters for the BKT model for our selected skill
and extract the records for a selected user.

In [ ]:
# Get params for BKT model
skill_name = 'Circle Graph'
p0 = df_params.loc[(skill_name, "prior", "default")].value
ps = df_params.loc[(skill_name, "slips", "default")].value
pg = df_params.loc[(skill_name, "guesses", "default")].value
pl = df_params.loc[(skill_name, "learns", "default")].value
pf = df_params.loc[(skill_name, "forgets", "default")].value

In [ ]:
# Get records for user 
user_id = 64525
records = df_preds.loc[
    (df_preds['user_id'] == user_id) & 
    (df_preds['skill_name'] == skill_name),
    ['correct', 'correct_predictions']
].values.tolist()

actuals = [r[0] for r in records]
ps,po = bkt_predict(p0,ps,pg,pl,pf,actuals)

print(po)

Now we can plot the observations versus the predictions from the model.

In [ ]:
# Plot the predictions versus the actual values for a user
def plot_user_predictions(actual, preds, user_id, skill_name):
    """
    actual: actual observation of the user
    preds: predicted probabilities of the model
    user_id: the user id (just for labeling in the title)
    skill_name: the skill name (for labeling in the title)
    """
    plt.figure(figsize=(8,4))
    
    # Plot predictions as a line
    plt.plot(preds, label='Prediction')
    
    # Plot actual values as points
    plt.scatter(range(len(actual)), actual, label='Actual')
    
    # Y-axis range
    plt.ylim(0, 1)
    
    plt.xlabel('Attempt Number')
    plt.ylabel('Value (0 or 1)')
    plt.title(f'Skill {skill_name}: Predictions vs Actual Observations for User {user_id}')
    plt.legend()
    plt.show()

In [ ]:
# Let's plot the model behavior for the selected user
plot_user_predictions(actuals, po, user_id, skill_name)

What does the prediction curve show? Does it capture the data well?

##### YOUR ANSWER HERE

### Task 2: RMSE by Skill
Next, we compute the overall RMSE across the six skills. For this, we will directly use the `df_preds`dataframe computed before.

In [ ]:
# Compute overall RMSE
# Hint: https://scikit-learn.org/stable/api/sklearn.metrics.html

rmse = # YOUR CODE HERE
print('Overall RMSE:', np.round(rmse, 3))

We hypothesize that performance of the model could depend on the skill. Some skills might be more difficult to predict than others. Next, we therefore compute the RMSE and AUC separately for each skill.

In [ ]:
# We first compute the metric for the specific skill "Circle Graph"
skill_1 = 'Circle Graph'

# Compute RMSE 
rmse = # YOUR CODE HERE

results = {'skill': skill_1, 'rmse': np.round(rmse, 3)}

print(results)

In [ ]:
# Compute RMSE per skill (for all skills)

def compute_skill_rmse(df_preds):
    """
    Compute RMSE (Root Mean Squared Error) per skill.

    Parameters
    ----------
    df_preds : pandas.DataFrame
        A dataframe containing predictions. Must contain at least:
            - 'skill_name': column identifying the skill/category
            - 'correct': the true labels (0/1 or numeric outcome)
            - 'correct_predictions': the predicted value for each item

    Returns
    -------
    pandas.DataFrame
        DataFrame with two columns:
            - 'skill_name'
            - 'rmse'
        Each row corresponds to one skill and its RMSE value.
    """

    # TODO: group df_preds by skill_name

    # TODO: for each group, compute RMS

    # TODO: convert the results into a DataFrame with:
    #   columns = ['skill_name', 'rmse']

    # TODO: return the resulting DataFrame
    pass


In [ ]:
df_skill_rmse = compute_skill_rmse(df_preds)
df_skill_rmse

Next, we visualize our results in a bar chart. We display the overall RMSE including standard deviations as well as the per skill performance metrics.

In [ ]:
def plot_skill_rmse(df_skill_rmse):
    """
    Plots:
      1) Overall RMSE across all skills with error bar (mean ± std)
      2) RMSE per skill (one bar per skill)

    Ensures both plots share the same y-axis scale.
    """

    # Mean and Std across skills
    mean_rmse = df_skill_rmse['rmse'].mean()
    std_rmse = df_skill_rmse['rmse'].std()

    # Prepare plotting data
    df_overall = pd.DataFrame({'label': ['All Skills'], 'rmse': [mean_rmse]})
    df_skill = df_skill_rmse.rename(columns={'skill_name': 'label', 'rmse': 'rmse'})

    # Create plots
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Plot 1: Overall RMSE with error bar
    sns.barplot(ax=axes[0], x='label', y='rmse', data=df_overall,  errorbar=None)
    axes[0].errorbar(x=[0], y=[mean_rmse], yerr=[std_rmse], fmt='none', capsize=5, linewidth=2, color='black')
    axes[0].set_title('Overall RMSE (mean ± std)')
    axes[0].set_xlabel('')
    axes[0].set_ylabel('RMSE')

    # Plot 2: RMSE per skill
    sns.barplot(ax=axes[1], x='label', y='rmse', data=df_skill)
    axes[1].set_title('RMSE per Skill')
    axes[1].set_xlabel('')
    axes[1].set_ylabel('RMSE')
    axes[1].tick_params(axis='x', rotation=90)

    # Make y-axis shared / equal
    y_max = max(df_skill['rmse'].max(), mean_rmse + std_rmse) * 1.1
    axes[0].set_ylim(0, y_max)
    axes[1].set_ylim(0, y_max)

    plt.tight_layout()
    plt.show()

plot_skill_rmse(df_skill_rmse)

Interpret the results. What do you observe?

##### YOUR ANSWER HERE

## Task 3: Analyzing Students' Learning Activity

We have now looked into BKT model predictions for a single student as well as into model error per skill. Next, we will use the BKT model predictions to analyze student behavior for each skill.

We will now create a function that computes that averages over the (predicted) success rate of all users at a given opportunity.

In [ ]:
def avg_y_by_x(x, y):
    '''
    Compute average learning curve and number of students over the number of opportunities. 
    x is the number of opportunities.
    y the success rates of the users (can be predicted success rate or true success rate).
    '''
    # Transform lists into arrays
    x = np.array(x)
    y = np.array(y)

    # Sort the integer id representing the number of opportunities in increasing order
    xs = sorted(list(set(x)))

    # Supporting lists to store the:
    # - xv: integer identifier of the number of opportunities
    # - yv: average value across students at that number of opportunities
    # - lcb and ucb: lower and upper confidence bound
    # - n_obs: number of observartions present at that number of opportunities (on per-skill plots, it is the #students)
    xv, yv, lcb, ucb, n_obs = [], [], [], [], []

    # For each integer identifier of the number of opportunities 0, ...
    for v in xs:
        ys = [y[i] for i, e in enumerate(x) if e == v] # We retrieve the values for that integer identifier
        if len(ys) > 0: 
            xv.append(v) # Append the integer identifier of the number of opportunities
            yv.append(sum(ys) / len(ys)) # Append the average value across students at that number of opportunities
            n_obs.append(len(ys)) # Append the number of observartions present at that number of opportunities

            
            # Prepare data for confidence interval computation
            unique, counts = np.unique(ys, return_counts=True)
            counts = dict(zip(unique, counts))

            if 0 not in counts:
                counts[0] = 0
            if 1 not in counts:
                counts[1] = 0

            # Calculate the 95% confidence intervals
            ci = sc.stats.beta.interval(0.95, 0.5 + counts[0], 0.5 + counts[1])
            lcb.append(ci[0])
            ucb.append(ci[1])

    return xv, yv, lcb, ucb, n_obs

Then, we create a function for plotting learning curve.

In [ ]:
# Rename the dataframe columns as per instructions
df_preds.columns = ['user_id', 'skill_name', 'y_true', 'y_pred_bkt']
predictions = df_preds

def plot_learning_curve(skill_name):
    '''
    Plot learning curve using BKT model for skill `skill_name`,
    showing only the learning curve (no histogram).
    '''

    preds = predictions[predictions['skill_name'] == skill_name]

    xp = []
    yp = {}
    for col in preds.columns:
        if 'y_' in col:
            yp[col] = []

    for user_id in preds['user_id'].unique():
        user_preds = preds[preds['user_id'] == user_id]
        xp += list(np.arange(len(user_preds)))
        for col in preds.columns:
            if 'y_' in col:
                yp[col] += user_preds[col].tolist()

    fig, ax = plt.subplots(1, 1, figsize=(6,4))

    lines = []
    for col in preds.columns:
        if 'y_' in col:
            x, y, lcb, ucb, n_obs = avg_y_by_x(xp, yp[col])
            if col == 'y_true':
                ax.fill_between(x, lcb, ucb, alpha=.1)
            model_line, = ax.plot(x, y, label=col)
            lines.append(model_line)

    ax.set_title(skill_name)
    ax.set_ylabel('Error')
    ax.set_ylim(0, 1)
    ax.set_xlim(0, None)
    ax.set_xlabel('#Opportunities')
    ax.legend(handles=lines)

    plt.tight_layout()
    plt.show()


Finally, we display and interpret the obtained curves for the selected six skills.

In [ ]:
# Plot the learning curves for all skills using the function 'plot_learning_curve' defined above

# YOUR CODE HERE

Analyse the skills. What behaviors do you observe? What can you say about different skills and average student progress? Do BKT predictions reflect true patterns in the data?

##### YOUR ANSWER HERE

Do you notice similarities in confidence bounds over time? What might be the cause of such patterns? Propose a hypothesis and verify it by visualizing the data using the function below.

##### YOUR HYPOTHESIS

In [ ]:
def plot_opportunity_histogram(skill_name):
    """
    Compute and plot the opportunity histogram for a given skill, using the same
    data processing as in plot_learning_curve.
    """

    preds = predictions[predictions['skill_name'] == skill_name]

    # Rebuild xp and yp just like in plot_learning_curve
    xp = []
    yp = {}
    for col in preds.columns:
        if 'y_' in col:
            yp[col] = []

    for user_id in preds['user_id'].unique():
        user_preds = preds[preds['user_id'] == user_id]
        xp += list(np.arange(len(user_preds)))
        for col in preds.columns:
            if 'y_' in col:
                yp[col] += user_preds[col].tolist()

    # Use y_true for n_obs (same as original behavior)
    x, y, lcb, ucb, n_obs = avg_y_by_x(xp, yp['y_true'])

    # --- Plot Histogram ---
    n_obs = np.asarray(n_obs, dtype=float)
    x_positions = np.arange(len(n_obs))

    plt.figure(figsize=(6,3))
    plt.bar(x_positions, n_obs)

    plt.xlabel('#Opportunities')
    plt.ylabel('#Observations')
    plt.title(f'Observations per Opportunity Index — {skill_name}')

    plt.ylim(0, np.max(n_obs) * 1.1)
    plt.xlim(-0.5, len(n_obs) - 0.5)


    plt.tight_layout()
    plt.show()


In [ ]:
plot_opportunity_histogram(skill_name) 

Number of student trials significantly drops over time which makes confidence interval increase.